# Time Series Forecasting mit rekurrenten neuronalen Netzen
Dieses Notebook entsteht im Rahmen des Moduls `Deep Learning` an der Fachhochschule Südwestfalen. Es beschäftigt sich mit dem Thema `Time Series Forecasting` und untersucht die Anwendung von rekurrenten neuronalen Netzen (RNN) auf Zeitreihendaten....

## 1. Einleitung

## 2. Was ist Time Series Forecasting (Zeitreihenvorhersage)?

Christian

Definition gemäß https://www.studysmarter.de/studium/mathematik-studium/statistik-studium/zeitreihenprognose/ (27.12.2024):<br>
**Zeitreihenprognose** ist eine statistische Methode zur Vorhersage zukünftiger Ereignisse oder Werte, indem historische Zeitreihendaten analysiert und Muster erkannt werden.

Beispiele:<br>
- Wetterdaten für zukünftiges Wetter (https://www.studysmarter.de/studium/mathematik-studium/statistik-studium/zeitreihenprognose/)
- Aktienkurse für zukünftige Aktienverläufe
- Passagierzahlen für zukünftige Passagiervorhersagen (https://data-science-crashkurs.de/chapters/kapitel_09.html)

Die Seite https://www.emft.fraunhofer.de/de/kompetenzen/systemloesungen-ki/ki-algorithmen-zeitreihenanalyse.html nennt zudem noch folgende mögliche Beispiele:
- Industrie 4.0 mit IIoT (Industrial Internet of Things)
- Logistik
- Finanzen
- Stromnetze
- Internetverkehr
- Umweltmonitoring
- Gesundheitsdaten

Außerdem werden auf der Seite typische Methoden der künstlichen Intelligenz inklusive grafiscgenannt, wie z.B.: Regression, Klassifikation, Anomalieerkennung, Vorhersage

Grafisches Beispiel einer Klassifikation:<br>
<img style = 'border: 5px solid #555' src="Pictures/classification-dt-mls-emft.jpg" width="30%" alt="Grafisches Beispiel einer Klassifikation"> 
<br>
Grafisches Beispiel einer Anomalieerkennung:<br>
<img style = 'border: 5px solid #555' src="Pictures/anomaly-dt-mls-emft.jpg" width="30%" alt="Grafisches Beispiel einer Anomalieerkennung">
<br>
Grafisches Beispiel einer Anomalieerkennung:<br>
<img style = 'border: 5px solid #555' src="Pictures/forecasting-dt-mls-emft.jpg" width="30%" alt="Grafisches Beispiel einer Vorhersage">

## 3. Was sind recurrent neural networks (Rekurrente neuronale Netzwerke)?

Ein rekurrentes oder auch rückgekoppeltes neuronales Netzwerk (RNN) ist ein künstliches neuronales Netzwerk, dass bei jeder Verarbeitung einer Einheit auch ein Ergebnis aus einer früheren Iteration erhält. Damit eignet es sich besonders für die Verarbeitung von sequentiellen Daten, wie z.B. Zeitreihendaten, Text oder Sprache.
Ein RNN funktioniert, indem es eine spezielle Ausgabe des vorherigen Schrittes, den hidden state oder auch state vector, als zusätzlichen Input für den nächsten Schritt verwendet. Das bedeutet, dass immer zwei Ausgaben generiert werden. Dadurch kann es sich an vorherige Iterationen erinnern und somit auch auf längere Abhängigkeiten in den Daten reagieren. Das eigentliche Ergebnis wird dabei nur im letzten Schritt betrachtet. [T1, S. 116ff]

<img style = 'border: 5px solid #555' src="Pictures/rnn-ablauf.png" width="30%" alt="Ablauf eines RNN">

[T1, S. 117]

Das Bild zeigt den sequentiellen Ablauf eines RNN. Dabei bezeichnet "t" den aktuellen Schritt. Diese Schritte könnten bei der Verarbeitung von Text z.B. einzelne Wörter oder Buchstaben sein. Der hidden state wird dabei in jedem Schritt aktualisiert und an den nächsten Schritt weitergegeben.

Die Verarbeitung einer Interation erfolgt dabei in drei Schritten die in folgendem Bild dargestellt sind:

<img style = 'border: 5px solid #555' src="Pictures/rnn-schritte.png" width="30%" alt="Ablauf einer RNN Iteration">

[T1, S. 119]

Der erste Schritt verbindet die beiden Eingangsvektoren zu einem einzigen Vektor. Anschließend wird der hidden State aktualisiert und im letzten Schritt wird der Ausgabewert berechnet.

Das vorangegangene Beispiel lässt sich mit wenigen Zeilen Code in PyTorch implementieren:

In [18]:
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden
    def initHidden(self):
        return Variable(torch.zeros(1, self.hidden_size))

In [20]:
toy_story_review = ['just', 'perfect', 'script', 'character', 'animation', 'this', 'manages', 'to', 'break', 'free']
input_size = 10
hidden_size = 15
output_size = 10

rnn = RNN(input_size, hidden_size, output_size)
hidden = rnn.initHidden()

for i in range(len(toy_story_review)):
    output, hidden = rnn(toy_story_review[i], hidden)

NameError: name 'torch' is not defined

Schleifenstruktur und Rückkopplung

Mathematische Formeln

Typen von RNNs
Einfaches, Probleme
LSTM
GRU

Quellen:
* T1: Mitchell, L. et al. (2019) Deep learning with Pytorch 1. x: implement deep learning techniques and neural network architecture variants using Python /. Second edition. Birmingham; Packt Publishing.


In [ ]:
Tim

## 4. Datensatz

## 5. Implementierung eines RNN

## 6. Aktuelle Modelle
Beispielmodelle Goldpreis oder Aktienpreis

## 7. Vergleich der Modelle
Vergleich mit transformer-netwerken?

## 8. Fazit
Eigenes Modell vs. verglichene Standardmodelle.